In [1]:
from datasets import load_dataset, DatasetDict, Dataset, load_from_disk
import pandas
from tqdm import tqdm

In [2]:
dataset_name = "presencesw/vinli_3_label"
label = 'entailment'
dataset = load_dataset(dataset_name)

In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['gold_label', 'sentence1', 'sentence2'],
        num_rows: 18282
    })
    test: Dataset({
        features: ['gold_label', 'sentence1', 'sentence2'],
        num_rows: 2264
    })
    dev: Dataset({
        features: ['gold_label', 'sentence1', 'sentence2'],
        num_rows: 2255
    })
})


In [4]:
def convert_entailment(exmaples):
    labels = exmaples['gold_label']
    labels = [f"not_{label}" if i != label else label for i in labels]
    exmaples['gold_label'] = labels
    return exmaples

In [5]:
# this step convert dataset to 2 labels
converted_dataset = dataset.map(convert_entailment, batched=True)

# check number samples in dataset train
converted_dataset['train']

Dataset({
    features: ['gold_label', 'sentence1', 'sentence2'],
    num_rows: 18282
})

In [6]:
# Divine the dataset to 2 sets `entailment` and `not entailment`
new_dataset_entail = converted_dataset.filter(lambda example: example['gold_label'] == label)['train']
new_dataset_not_entail = converted_dataset.filter(lambda example: example['gold_label'] == f'not_{label}')['train']

In [7]:
# convert dataset to pandas for next step
new_dataset_entail = Dataset.to_pandas(new_dataset_entail)
# duplicate dataset not entailment
new_dataset_not_entail1 = Dataset.to_pandas(new_dataset_not_entail)
new_dataset_not_entail2 = Dataset.to_pandas(new_dataset_not_entail)

In [8]:
# merge 2 dataset not entailment for the target
j = 0
for i in tqdm(range(len(new_dataset_not_entail1))):
    temp = set()
    # check = False
    # for j in range(len(new_dataset_not_entail2)):
    for index in range(100):
        new_j = j + index
        if new_j == len(new_dataset_not_entail1):
            j = new_j
            break
        if new_dataset_not_entail1.iloc[i]['sentence1'] != new_dataset_not_entail2.iloc[new_j]['sentence1']:
            j = new_j
            break
        temp.add(new_dataset_not_entail2.iloc[new_j]['sentence2'])
    # new_dataset_not_entail1.iloc[i]['sentence2'] = temp
    # temp = [val for val in temp]
    new_dataset_not_entail1.at[i, 'sentence2'] = temp

100%|██████████| 12188/12188 [00:01<00:00, 10877.98it/s]


In [9]:
# dataset_not_entail = new_dataset_not_entail1.drop_duplicates(subset=['sentence2'])
# print(len(dataset_not_entail))
dataset_not_entail = new_dataset_not_entail1[new_dataset_not_entail1['sentence2'] != set()]
print(len(dataset_not_entail))

3044


In [10]:
dataset_not_entail = dataset_not_entail.reset_index(drop=True)
dataset_not_entail

,gold_label,sentence1,sentence2
0,not_entailment,"Sau dự án điện mặt trời, Tập đoàn Trung Nam vừ...",{Tập đoàn Trung Nam có ý định bán toàn bộ cổ p...
1,not_entailment,"Phía Tập đoàn Trung Nam cho biết, với 64,9% cổ...",{Tuy giữ nhiều cổ phần hơn nhưng Tập đoàn Trun...
2,not_entailment,"Cách đây một tháng, tập đoàn này cũng bán 49% ...",{Việc sản xuất điện mặt trời trong hai năm qua...
3,not_entailment,"Dự án Nhà máy Điện gió Trung Nam, tổng số vốn ...",{Vốn đầu tư của Nhà máy Điện gió Trung Nam sẽ ...
4,not_entailment,"Từ nay tới năm 2028, dòng vốn đầu tư được dự b...",{Mặc cho những hoạt động đi lại còn nhiều hạn ...
...,...,...,...
3039,not_entailment,Ổ SSD dùng để khai thác tiền số Chia chỉ còn t...,{Ổ cứng đã được sử dụng để khai thác tiền số c...
3040,not_entailment,"Khi khai thác Chia, những ổ cứng dung lượng lớ...",{Tuổi thọ trung bình của những ổ cứng sử dụng ...
3041,not_entailment,Ổ cứng thường được hãng bảo hành từ ba đến năm...,{Việc có một hãng sản xuất ổ cứng đưa ra chính...
3042,not_entailment,"Ổ SSD được ""thợ đào"" Chia yêu thích vì tốc độ ...",{Tốc độ xử lý dữ liệu của ổ cứng SSD cao hơn g...


In [11]:
new_dataset_entail

,gold_label,sentence1,sentence2
0,entailment,"Sau dự án điện mặt trời, Tập đoàn Trung Nam vừ...",Công ty Hitachi Sustainable Energy có cổ phần ...
1,entailment,"Sau dự án điện mặt trời, Tập đoàn Trung Nam vừ...",Tập đoàn Trung Nam vừa thực hiện dự án điện nă...
2,entailment,"Phía Tập đoàn Trung Nam cho biết, với 64,9% cổ...",Tập đoàn Trung Nam nắm giữ phần lớn lượng cổ p...
3,entailment,"Phía Tập đoàn Trung Nam cho biết, với 64,9% cổ...",Tập đoàn Trung Nam là người có thể đưa ra quyế...
4,entailment,"Cách đây một tháng, tập đoàn này cũng bán 49% ...",Số cổ phần của tập đoàn Trung Nam tại Nhà máy ...
...,...,...,...
6089,entailment,Ổ cứng thường được hãng bảo hành từ ba đến năm...,Năm năm là mức thời gian bảo hành tối đa cho m...
6090,entailment,"Ổ SSD được ""thợ đào"" Chia yêu thích vì tốc độ ...",Ổ SSD có tốc độ xử lý cao hơn ổ HDD.
6091,entailment,"Ổ SSD được ""thợ đào"" Chia yêu thích vì tốc độ ...",Nhiều người dùng ổ cứng SSD để khai thác tiền ...
6092,entailment,"Chia là tiền điện tử đang ""sốt"" tại Trung Quốc.",Chia là một loại tiền ảo đang được rất nhiều n...


In [12]:
dataset_not_entail = dataset_not_entail.rename(columns={"sentence2": "negative"})
dataset_not_entail = dataset_not_entail.drop(columns="gold_label")

In [13]:
new_dataset_entail = new_dataset_entail.merge(dataset_not_entail, how="left", left_on="sentence1", right_on="sentence1")

In [14]:
new_dataset_entail

,gold_label,sentence1,sentence2,negative
0,entailment,"Sau dự án điện mặt trời, Tập đoàn Trung Nam vừ...",Công ty Hitachi Sustainable Energy có cổ phần ...,{Tập đoàn Trung Nam có ý định bán toàn bộ cổ p...
1,entailment,"Sau dự án điện mặt trời, Tập đoàn Trung Nam vừ...",Tập đoàn Trung Nam vừa thực hiện dự án điện nă...,{Tập đoàn Trung Nam có ý định bán toàn bộ cổ p...
2,entailment,"Phía Tập đoàn Trung Nam cho biết, với 64,9% cổ...",Tập đoàn Trung Nam nắm giữ phần lớn lượng cổ p...,{Tuy giữ nhiều cổ phần hơn nhưng Tập đoàn Trun...
3,entailment,"Phía Tập đoàn Trung Nam cho biết, với 64,9% cổ...",Tập đoàn Trung Nam là người có thể đưa ra quyế...,{Tuy giữ nhiều cổ phần hơn nhưng Tập đoàn Trun...
4,entailment,"Cách đây một tháng, tập đoàn này cũng bán 49% ...",Số cổ phần của tập đoàn Trung Nam tại Nhà máy ...,{Việc sản xuất điện mặt trời trong hai năm qua...
...,...,...,...,...
6121,entailment,"Ổ SSD được ""thợ đào"" Chia yêu thích vì tốc độ ...",Nhiều người dùng ổ cứng SSD để khai thác tiền ...,{Tốc độ xử lý dữ liệu của ổ cứng SSD cao hơn g...
6122,entailment,"Chia là tiền điện tử đang ""sốt"" tại Trung Quốc.",Chia là một loại tiền ảo đang được rất nhiều n...,{Đồng tiền điện tử Chia bị cấm lưu hành tại Tr...
6123,entailment,"Chia là tiền điện tử đang ""sốt"" tại Trung Quốc.",Chia là một loại tiền ảo đang được rất nhiều n...,"{Sau nhiều năm ra mắt, cuối cùng thì tiền điện..."
6124,entailment,"Chia là tiền điện tử đang ""sốt"" tại Trung Quốc.","Ở Trung Quốc, rất nhiều người đang cố gắng đào...",{Đồng tiền điện tử Chia bị cấm lưu hành tại Tr...


In [15]:
# temp = []
# for index1 in tqdm(range(len(new_dataset_entail))):
#     # print(new_dataset_entail.iloc[index1]['sentence1'])
#     for index2 in range(len(dataset_not_entail)):
#         if new_dataset_entail.iloc[index1]['sentence1'] == dataset_not_entail.iloc[index2]['sentence1']:
#             temp.append(dataset_not_entail.iloc[index2]['sentence2'])
#             break

In [16]:
# print(len(temp))

In [17]:
# new_dataset_entail['negative'] = temp
# new_dataset_entail

In [18]:
new_dataset = Dataset.from_pandas(new_dataset_entail)
new_dataset = new_dataset.rename_column("sentence1", "anchor")
new_dataset = new_dataset.rename_column("sentence2", "positive")
new_dataset

Dataset({
    features: ['gold_label', 'anchor', 'positive', 'negative'],
    num_rows: 6126
})

In [19]:
def convert_dataset(example):
    # print(example['negative'])
    number = len(example['negative'][0])
    # print(gold_label)
    gold_label = example['gold_label'] * number
    anchor = example['anchor'] * number
    positive = example['positive'] * number
    return {'gold_label': gold_label, 'anchor': anchor, 
            'positive': positive, 'negative': example['negative'][0]}

        
new_dataset = new_dataset.map(
    lambda example: convert_dataset(example),                   
    remove_columns=['gold_label', 'anchor', 'positive', 'negative'],
    batched=True,
    batch_size=1
)

Map:   0%|          | 0/6126 [00:00<?, ? examples/s]

In [20]:
new_dataset

Dataset({
    features: ['gold_label', 'anchor', 'positive', 'negative'],
    num_rows: 24536
})

In [21]:
print(['']*5)

['', '', '', '', '']


In [22]:
def add_columns(examples):
    # print(len(examples))
    length = len(examples['gold_label'])
    examples['negative'] = [''] * length
    return examples

In [23]:
converted_dataset = converted_dataset.rename_column("sentence1", "anchor")
converted_dataset = converted_dataset.rename_column("sentence2", "positive")

In [24]:
converted_dataset = converted_dataset.map(add_columns, batched=True)

In [25]:
all_data = DatasetDict()
all_data['train'] = new_dataset
all_data['validation'] = converted_dataset['dev']
all_data['test'] = converted_dataset['test']

In [26]:
all_data.push_to_hub(f"{dataset_name}_{label}")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/25 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/606 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/presencesw/vinli_3_label_entailment/commit/c8ca77e7edf8b047fc49b633446c1106d3437d81', commit_message='Upload dataset', commit_description='', oid='c8ca77e7edf8b047fc49b633446c1106d3437d81', pr_url=None, pr_revision=None, pr_num=None)